# 1.0 Data Wrangling <a name="1.1_datawrangling"></a>


## Contents <a name="contents"></a>

* [1 Data Wrangling](#1.1_datawrangling)

    * [Contents](#contents)


* [1.2 Introduction](#1.2_introduction)

    * [1.2.1 Problem Recap](#1.2.1_recap)

    * [1.2.2 Data Wrangling Introduction](#1.2.2_wrangling)
    

* [1.3 Imports](#1.3_imports)


* [1.4 Load Data](#1.4_loaddata)


* [1.5 Fashion Dataset](#1.5_exploredata)

    * [1.5.1 Fashion data features](#1.5.1_feat)
    
    * [1.5.2 Fashion missing data](#1.5.2_f_missing)


* [1.6 Electronics dataset](#1.6_electronics)
    
    * [1.6.1 Electronics data features](#1.6.1_feat)

    * [1.6.2 Electronics missing data](#1.6.2_e_missing)
        

* [1.7 Home/kitchen dataset](#1.7_h/k)
    
    * [1.7.1 Home/kitchen data features](#1.7.1_h/k_exp)
        
    * [1.7.2 Home/kitchen missing data](#1.7.2_h/k_missing)
            
        
* [1.8 Save Data](#1.8)
    
    * [1.8.1 Creating a parsing function](#1.8.1)
    
    * [1.8.2 Fashion CSV](#1.8.2)
        
    * [1.8.3 Electronics CSV](#1.8.3)
        
    * [1.8.4 Home/Kitchen CSV](#1.8.4)


* [1.9 Summary](#1.9_summary)

## 1.1 Data Wrangling 

## 1.2 Introduction<a name="1.2_introduction"></a>
In this notebook, we are going to primarily be focusing on loading the data, examining missing and duplicated values, and dropping unneccesary features. We may do a small amount of data cleaning, but we do not want to prematurely drop features that we may later need.

### 1.2.1 Problem Recap <a name="1.2.1_recap"></a>

Using customer text data about amazon products, we will build, evaluate and compare models to estimate the probability that a given text review can be classified as “positive”, “neutral”, or “negative”.

Our goal is to build a text classifier using Amazon product review data which can be used to analyze customer sentiment which does not have accompanying numeric data.

### 1.2.2 Data Wrangling Introduction <a name="1.2.2_wrangling"></a>
We will read in the data, remove unnecessary columns/features, and then save it in an easier-to-work-with tabular format (CSV). 

## 1.3 Imports <a name="1.3_imports"></a>

In [1]:
import json
import csv
import pandas as pd

## 1.4 Load Data<a name="1.4_loaddata"></a>
Datasets are sourced from:

[Jianmo Ni, Jiacheng Li, Julian McAuley
Empirical Methods in Natural Language Processing (EMNLP), 2019](https://nijianmo.github.io/amazon/index.html)

The data is split up by product category. 

Our categories will be:

#### Fashion (883,636 reviews)

#### Electronics (20,994,353 reviews)

#### Home and Kitchen (21,928,568 reviews)

The data is available as compressed JSON files. We will use pandas to load the fashion data and determine which features to keep or drop and which features need preliminary cleaning.

Because of how large the Electronics and Home and Kitchen datasets are (approx. 10 GB each), we'll use the fashion dataset to explore the features and build a set of functions to parse each of the three category files. We will stream the JSON data, drop the values/features we do not want to keep, and save to separate CSV files.

In [2]:
!ls -hl data/raw

ls: cannot access 'data/raw': No such file or directory


## 1.5 Explore the Fashion data <a name="1.5_exploredata"></a>

Let's take a look at the top of the **Fashion** dataset

In [3]:
fashion_head = pd.read_json("../data/raw/AMAZON_FASHION.json", lines=True, nrows=200)

In [4]:
fashion_head.head(3)

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,5,True,"10 20, 2014",A1D4G1SNUZWQOT,7106116521,Tracy,Exactly what I needed.,perfect replacements!!,1413763200,NaN,NaN,NaN
1,2,True,"09 28, 2014",A3DDWDH9PX2YX2,7106116521,Sonja Lau,"I agree with the other review, the opening is ...","I agree with the other review, the opening is ...",1411862400,3.0,NaN,NaN
2,4,False,"08 25, 2014",A2MWC41EW7XL15,7106116521,Kathleen,Love these... I am going to order another pack...,My New 'Friends' !!,1408924800,NaN,NaN,NaN


In [5]:
fashion_head.dtypes

overall             int64
verified             bool
reviewTime         object
reviewerID         object
asin               object
reviewerName       object
reviewText         object
summary            object
unixReviewTime      int64
vote              float64
style              object
image              object
dtype: object

### 1.5.1 Fashion data features <a name="1.5.1_feat"></a>

The **overall** feature is the number of stars, which we'll be using for our categorical target feature (positive/neutral/negative). It's an integer from 1-5. 

The **verified** feature is a boolean and determines whether Amazon can confirm the user purchased the item they're reviewing (not a free/discounted reviewed item).

The **reviewTime** is in a date string format, and we'll drop it and keep the **unixReviewTime** instead.

The **reviewerID** and **reviewerName** have similar information and we can keep just one (the ID should be fine).

The **asin** is the specific product ID. We could keep it if we want to get item metadata later and use that for additional exploratory analysis (ie. price, product tags, product subcategory).

The **summary** is a user provided (or amazon generated) short item summary.

The **vote** column seems to indicate how many users found a review useful.

### Important features:

The **reviewText** will be the main input feature/features we will be using for our sentiment prediction and modelling.

Our target feature will be based on the **overall** column. We will be using the star value for each review to convert the data to Negative/Neutral/Positive categories. This will be the feature that our model trains and predicts with.

Let's check the **style** column.

In [6]:
fashion_head[~(fashion_head["style"].isna())]

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
7,3,True,"09 22, 2013",A1BB77SEBQT8VX,B00007GDFV,Darrow H Ankrum II,mother - in - law wanted it as a present for h...,bought as a present,1379808000,NaN,{'Color:': ' Black'},NaN
8,3,True,"07 17, 2013",AHWOW7D1ABO9C,B00007GDFV,rosieO,"Item is of good quality. Looks great, too. But...",Buxton heiress collection,1374019200,NaN,{'Color:': ' Black'},NaN
9,3,True,"04 13, 2013",AKS3GULZE0HFC,B00007GDFV,M. Waltman,I had used my last el-cheapo fake leather ciga...,Top Clasp Broke Within 3 days!,1365811200,NaN,{'Color:': ' Black'},NaN
10,4,True,"03 9, 2013",A38NS6NF6WPXS,B00007GDFV,BTDoxies,This brand has been around a long time and you...,BUXTON QUALITY!,1362787200,NaN,{'Color:': ' Black'},NaN
11,2,True,"01 27, 2013",A1KOKO3HTSAI1H,B00007GDFV,Robin Howard,I smoke 100's and these are NOT made for them....,Buxton Heiress Collection Black Leather Cigare...,1359244800,NaN,{'Color:': ' Black'},NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
175,5,True,"08 28, 2016",A3U8IMV1FKI5Y3,B00008JPRZ,SPB,Love shirt and even makes me look good!,Love the shirt!!!,1472342400,NaN,"{'Size:': ' 16.5 - 34', 'Color:': ' White'}",NaN
176,5,True,"05 25, 2016",A21CTBULJC61N1,B00008JPRZ,Harrycarnival,great shirt,Five Stars,1464134400,NaN,"{'Size:': ' 17.5 - 37', 'Color:': ' White'}",NaN
193,5,False,"03 2, 2006",A3JGLFGCB7KZQX,B0000AWXMM,DG,"It was exactly as described. Beautiful ring, ...",BEAUTIFUL RING!!!! NO REGRETS WHATSOEVER,1141257600,NaN,"{'Size:': ' 12', 'Metal Type:': ' yellow-gold'}",NaN
194,4,False,"08 22, 2005",A179HMPWBPTJ2I,B0000AWXMM,Leana J. Kemp,This is a nice ring. It is simple and elegant...,White Gold Ring,1124668800,6.0,"{'Size:': ' 10', 'Metal Type:': ' white-gold'}",NaN


The **style** column seems to have sizing and color info but is missing for most reviews.

### 1.5.2 Fashion missing data <a name="1.5.2_f_missing"></a>

In [7]:
fashion = pd.read_json("../data/raw/AMAZON_FASHION.json", lines=True, chunksize=100000)

Using a json_reader iterator from Pandas, we can iterate through the data rather than loading the entire dataset into memory


In [8]:
missing_stats=[]
for chunk in fashion:
    missing_stats.append(chunk.isna().sum())

In [9]:
missing_vals = pd.DataFrame(missing_stats)

In [10]:
missing_vals

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,0,0,0,0,0,6,109,39,0,90595,28037,98423
1,0,0,0,0,0,18,108,61,0,89490,49765,97805
2,0,0,0,0,0,13,173,68,0,89152,44147,95963
3,0,0,0,0,0,14,172,77,0,86690,41255,94941
4,0,0,0,0,0,6,93,26,0,92429,78710,98412
5,0,0,0,0,0,5,123,49,0,93244,88410,97702
6,0,0,0,0,0,7,174,78,0,92668,87838,96291
7,0,0,0,0,0,11,152,76,0,92517,87214,95966
8,0,0,0,0,0,12,129,59,0,76951,73691,79326


In [11]:
missing_vals.sum()

overall                0
verified               0
reviewTime             0
reviewerID             0
asin                   0
reviewerName          92
reviewText          1233
summary              533
unixReviewTime         0
vote              803736
style             579067
image             854829
dtype: int64

We have no missing values in our **overall** column, which is excellent. The **vote**, **style** and **image** columns have a lot of missing values, which is fine as we'll be dropping those columns. There are not many missing **reviewText** values.

Our final column/feature list to keep will be:

* **overall**
* **reviewerID**
* **verified**
* **asin**
* **reviewText**
* **summary**
* **unixReviewTime**

In [12]:
cols_to_keep = ["overall",  "verified", "reviewerID", "asin", "reviewText", "summary", "unixReviewTime"]

Let's do a quick preview to make sure our column list is correct.

In [13]:
fashion_head[cols_to_keep].head()

,overall,verified,reviewerID,asin,reviewText,summary,unixReviewTime
0,5,True,A1D4G1SNUZWQOT,7106116521,Exactly what I needed.,perfect replacements!!,1413763200
1,2,True,A3DDWDH9PX2YX2,7106116521,"I agree with the other review, the opening is ...","I agree with the other review, the opening is ...",1411862400
2,4,False,A2MWC41EW7XL15,7106116521,Love these... I am going to order another pack...,My New 'Friends' !!,1408924800
3,2,True,A2UH2QQ275NV45,7106116521,too tiny an opening,Two Stars,1408838400
4,3,False,A89F3LQADZBS5,7106116521,Okay,Three Stars,1406419200


## 1.6 Electronics dataset <a name="1.6_electronics"></a>


In [14]:
elec_head = pd.read_json("../data/raw/Electronics.json", lines=True, nrows=1000)

In [15]:
elec_head.head()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,5,True,"07 17, 2002",A1N070NS9CJQ2I,60009810,{'Format:': ' Hardcover'},Teri Adams,This was the first time I read Garcia-Aguilera...,Hit The Spot!,1026864000,NaN,NaN
1,5,False,"07 6, 2002",A3P0KRKOBQK1KN,60009810,{'Format:': ' Hardcover'},Willa C.,"As with all of Ms. Garcia-Aguilera's books, I ...",one hot summer is HOT HOT HOT!,1025913600,NaN,NaN
2,5,False,"07 3, 2002",A192HO2ICJ75VU,60009810,{'Format:': ' Hardcover'},Kit,I've not read any of Ms Aguilera's works befor...,One Hot Summer,1025654400,2.0,NaN
3,4,False,"06 30, 2002",A2T278FKFL3BLT,60009810,{'Format:': ' Hardcover'},Andres,This romance novel is right up there with the ...,I love this book!,1025395200,3.0,NaN
4,5,False,"06 28, 2002",A2ZUXVTW8RXBXW,60009810,{'Format:': ' Hardcover'},John,Carolina Garcia Aguilera has done it again. S...,One Hot Book,1025222400,NaN,NaN


### 1.6.1 Electronics data features <a name="1.6.1_feat"></a>



In [16]:
elec_head.dtypes

overall             int64
verified             bool
reviewTime         object
reviewerID         object
asin                int64
style              object
reviewerName       object
reviewText         object
summary            object
unixReviewTime      int64
vote              float64
image              object
dtype: object

It looks like our column names and data is pretty similar. Let's check and make sure the columns we wish to keep have the same names.

In [17]:
elec_head[cols_to_keep]

,overall,verified,reviewerID,asin,reviewText,summary,unixReviewTime
0,5,True,A1N070NS9CJQ2I,60009810,This was the first time I read Garcia-Aguilera...,Hit The Spot!,1026864000
1,5,False,A3P0KRKOBQK1KN,60009810,"As with all of Ms. Garcia-Aguilera's books, I ...",one hot summer is HOT HOT HOT!,1025913600
2,5,False,A192HO2ICJ75VU,60009810,I've not read any of Ms Aguilera's works befor...,One Hot Summer,1025654400
3,4,False,A2T278FKFL3BLT,60009810,This romance novel is right up there with the ...,I love this book!,1025395200
4,5,False,A2ZUXVTW8RXBXW,60009810,Carolina Garcia Aguilera has done it again. S...,One Hot Book,1025222400
...,...,...,...,...,...,...,...
995,5,True,A107S4MT25VXQ5,594481902,Perfect...Nook brand so exactly what was neede...,Juat what I needed...,1399507200
996,4,True,A1T8TP4OHT5T13,594481902,This cord helps me have two locations to plug ...,Extra Cord,1399420800
997,5,True,A3HICVLF4PFFMN,594481902,"bought for a spare for my 9"" Nook HD and it fi...",great fit,1399248000
998,5,True,AY0BL2JD0GAFT,594481902,Lost my charger. Found this one for less than ...,Lost the.charger.for.my Nook!,1399248000


### 1.6.2 Electronics missing data <a name=1.6.2_e_missing></a>

In [18]:
elect = pd.read_json("../data/raw/Electronics.json", lines=True, chunksize=100000)

Using a json_reader iterator from Pandas, we'll iterate through the dataset to get missing value summaries, like we did with the fashion dataset.

In [19]:
missing_stats=[]
for chunk in elect:
    missing_stats.append(chunk.isna().sum())

In [20]:
missing_vals = pd.DataFrame(missing_stats)

In [21]:
missing_vals.sum()

overall                  0
verified                 0
reviewTime               0
reviewerID               0
asin                     0
style             10497616
reviewerName          1713
reviewText            9684
summary               4754
unixReviewTime           0
vote              18300976
image             20645630
dtype: int64

Similarly to the fashion dataset, the style, vote and image columns have lots of missing values but we'll be dropping those columns. The two important columns, **overall** and **reviewText** have 0 and 9684 missing values respectively out of about 20 million reviews.

## 1.7 Home/kitchen dataset <a name="1.7_h/k"></a>            

In [22]:
hk_head = pd.read_json("../data/raw/Home_and_Kitchen.json", lines=True, nrows=200)

In [23]:
hk_head.head()

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image
0,5,2.0,True,"08 31, 2010",A3NSN9WOX8470M,0006564224,mmm,"I don't use these for their original use, and ...",Many uses...,1283212800,NaN,NaN
1,5,2.0,True,"04 2, 2010",A2AMX0AJ2BUDNV,0006564224,John R. Welch,"Seems a bit expensive for a plastic bottle, bu...",Dispenser bottle,1270166400,NaN,NaN
2,5,NaN,True,"11 5, 2015",A8LUWTIPU9CZB,0560467893,Linda Fahner,"Great product, love it!!",Five Stars,1446681600,NaN,NaN
3,4,4.0,True,"10 29, 2015",AABKIIHAL0L66,0560467893,TheBlueChain,This is a sturdy floating corner shelf! We mo...,"Sturdy Shelf, Poor Installation Instructions",1446076800,NaN,NaN
4,3,NaN,True,"09 9, 2015",A3DA0KIQ5OBK5C,0560467893,angelaarden,I purchased 4 of these shelves. they look grea...,Look great - one bad one...,1441756800,NaN,NaN


In [24]:
hk_head.dtypes

overall             int64
vote              float64
verified             bool
reviewTime         object
reviewerID         object
asin               object
reviewerName       object
reviewText         object
summary            object
unixReviewTime      int64
style              object
image              object
dtype: object

### 1.7.1 Home/kitchen data features <a name="1.7.1_h/k_exp"></a>

In [25]:
hk_head

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image
0,5,2.0,True,"08 31, 2010",A3NSN9WOX8470M,0006564224,mmm,"I don't use these for their original use, and ...",Many uses...,1283212800,NaN,NaN
1,5,2.0,True,"04 2, 2010",A2AMX0AJ2BUDNV,0006564224,John R. Welch,"Seems a bit expensive for a plastic bottle, bu...",Dispenser bottle,1270166400,NaN,NaN
2,5,NaN,True,"11 5, 2015",A8LUWTIPU9CZB,0560467893,Linda Fahner,"Great product, love it!!",Five Stars,1446681600,NaN,NaN
3,4,4.0,True,"10 29, 2015",AABKIIHAL0L66,0560467893,TheBlueChain,This is a sturdy floating corner shelf! We mo...,"Sturdy Shelf, Poor Installation Instructions",1446076800,NaN,NaN
4,3,NaN,True,"09 9, 2015",A3DA0KIQ5OBK5C,0560467893,angelaarden,I purchased 4 of these shelves. they look grea...,Look great - one bad one...,1441756800,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
195,5,NaN,True,"09 1, 2015",A1PNQ4F0UC9V4,1605161810,party,I like this and thank you.,Five Stars,1441065600,NaN,NaN
196,5,NaN,True,"08 14, 2015",AC1W8G7HU9VH8,1605161810,Jeanette Dodds,My mom loves it! Delivery was exactly on time....,Five Stars! Beautiful!,1439510400,NaN,[https://images-na.ssl-images-amazon.com/image...
197,5,NaN,True,"07 18, 2015",AFY118HH4OG7F,1605161810,GothicPoohBear,good quality!,good quality,1437177600,NaN,NaN
198,5,NaN,True,"06 19, 2015",AG6BH7V520TJ5,1605161810,Deb,Beautiful!,Love it!,1434672000,NaN,NaN


Looks like the same columns names. Let's make sure.

In [26]:
hk_head[cols_to_keep]

,overall,verified,reviewerID,asin,reviewText,summary,unixReviewTime
0,5,True,A3NSN9WOX8470M,0006564224,"I don't use these for their original use, and ...",Many uses...,1283212800
1,5,True,A2AMX0AJ2BUDNV,0006564224,"Seems a bit expensive for a plastic bottle, bu...",Dispenser bottle,1270166400
2,5,True,A8LUWTIPU9CZB,0560467893,"Great product, love it!!",Five Stars,1446681600
3,4,True,AABKIIHAL0L66,0560467893,This is a sturdy floating corner shelf! We mo...,"Sturdy Shelf, Poor Installation Instructions",1446076800
4,3,True,A3DA0KIQ5OBK5C,0560467893,I purchased 4 of these shelves. they look grea...,Look great - one bad one...,1441756800
...,...,...,...,...,...,...,...
195,5,True,A1PNQ4F0UC9V4,1605161810,I like this and thank you.,Five Stars,1441065600
196,5,True,AC1W8G7HU9VH8,1605161810,My mom loves it! Delivery was exactly on time....,Five Stars! Beautiful!,1439510400
197,5,True,AFY118HH4OG7F,1605161810,good quality!,good quality,1437177600
198,5,True,AG6BH7V520TJ5,1605161810,Beautiful!,Love it!,1434672000


### 1.7.2 Home/kitchen missing <a name="1.7.2_h/k_missing"></a>

In [27]:
hk = pd.read_json("../data/raw/Home_and_Kitchen.json", lines=True, chunksize=100000)

Using a json_reader iterator from Pandas, we'll iterate through the dataset to get missing value summaries, like we did with the fashion dataset.

In [28]:
missing_stats=[]
for chunk in hk:
    missing_stats.append(chunk.isna().sum())

In [29]:
missing_vals = pd.DataFrame(missing_stats)

In [30]:
missing_vals.sum()

overall                  0
vote              19117548
verified                 0
reviewTime               0
reviewerID               0
asin                     0
reviewerName          1160
reviewText           14866
summary               5781
unixReviewTime           0
style              9785019
image             21296935
dtype: int64

Out of about 22 million reviews, we have no missing **overall** ratings and only 14,866 missing **reviewText**.

## 1.8 Save Data <a name="1.8"></a>

### 1.8.1 Creating a parsing function <a name="1.8.1"></a>

In [31]:
def df_cleaner(temp_df_chunk: pd.DataFrame,
               columns_to_keep=cols_to_keep) -> pd.DataFrame:
    
    """This function will take in a chunk of our dataset as a dataframe.
       It will return only the columns we wish.
       
       Takes:
           temp_df_chunk: a dataframe to be edited
           columns_to_keep: the list of columns we wish to keep from that df          
       Returns:
           clean_df: a smaller df with drop_by_row NaNs dropped and only a subset of original columns
       """
  
    return temp_df_chunk[columns_to_keep]

Let's do a quick test.

In [32]:
df_cleaner(hk_head)

,overall,verified,reviewerID,asin,reviewText,summary,unixReviewTime
0,5,True,A3NSN9WOX8470M,0006564224,"I don't use these for their original use, and ...",Many uses...,1283212800
1,5,True,A2AMX0AJ2BUDNV,0006564224,"Seems a bit expensive for a plastic bottle, bu...",Dispenser bottle,1270166400
2,5,True,A8LUWTIPU9CZB,0560467893,"Great product, love it!!",Five Stars,1446681600
3,4,True,AABKIIHAL0L66,0560467893,This is a sturdy floating corner shelf! We mo...,"Sturdy Shelf, Poor Installation Instructions",1446076800
4,3,True,A3DA0KIQ5OBK5C,0560467893,I purchased 4 of these shelves. they look grea...,Look great - one bad one...,1441756800
...,...,...,...,...,...,...,...
195,5,True,A1PNQ4F0UC9V4,1605161810,I like this and thank you.,Five Stars,1441065600
196,5,True,AC1W8G7HU9VH8,1605161810,My mom loves it! Delivery was exactly on time....,Five Stars! Beautiful!,1439510400
197,5,True,AFY118HH4OG7F,1605161810,good quality!,good quality,1437177600
198,5,True,AG6BH7V520TJ5,1605161810,Beautiful!,Love it!,1434672000


### 1.8.2 Fashion CSV <a name="1.8.2"></a>

In [33]:
fashion = pd.read_json("../data/raw/AMAZON_FASHION.json", lines=True, chunksize=1_000_000)

In [34]:
for i, chunk in enumerate(fashion):
    df_cleaner(chunk).to_parquet(path=f"../data/edited/fashion{i}.parquet")

In [35]:
ls ../data/edited/

fashion0.parquet


In [36]:
pd.read_parquet("../data/edited/fashion0.parquet").head()

,overall,verified,reviewerID,asin,reviewText,summary,unixReviewTime
0,5,True,A1D4G1SNUZWQOT,7106116521,Exactly what I needed.,perfect replacements!!,1413763200
1,2,True,A3DDWDH9PX2YX2,7106116521,"I agree with the other review, the opening is ...","I agree with the other review, the opening is ...",1411862400
2,4,False,A2MWC41EW7XL15,7106116521,Love these... I am going to order another pack...,My New 'Friends' !!,1408924800
3,2,True,A2UH2QQ275NV45,7106116521,too tiny an opening,Two Stars,1408838400
4,3,False,A89F3LQADZBS5,7106116521,Okay,Three Stars,1406419200


#### Looks like our function and loop worked great. It will take a lot longer for the two larger files.

### 1.8.3 Electronics CSV <a name="1.8.3"></a> 
    

In [37]:
%reset -f

In [38]:
import json
import csv
import pandas as pd

In [39]:
cols_to_keep = ["overall",  "verified", "reviewerID", "asin", "reviewText", "summary", "unixReviewTime"]

def df_cleaner(temp_df_chunk: pd.DataFrame,
               columns_to_keep=cols_to_keep) -> pd.DataFrame:
    return temp_df_chunk[columns_to_keep]

In [40]:
elect = pd.read_json("../data/raw/Electronics.json", lines=True, chunksize=1_000_000)

In [41]:
for i, chunk in enumerate(elect):
    df_cleaner(chunk).to_parquet(path=f"../data/edited/electronics{i}.parquet")

In [42]:
ls ../data/edited/

electronics0.parquet   electronics17.parquet  electronics5.parquet
electronics10.parquet  electronics18.parquet  electronics6.parquet
electronics11.parquet  electronics19.parquet  electronics7.parquet
electronics12.parquet  electronics1.parquet   electronics8.parquet
electronics13.parquet  electronics20.parquet  electronics9.parquet
electronics14.parquet  electronics2.parquet   fashion0.parquet
electronics15.parquet  electronics3.parquet
electronics16.parquet  electronics4.parquet


In [43]:
pd.read_parquet("../data/edited/electronics10.parquet").head(20)

,overall,verified,reviewerID,asin,reviewText,summary,unixReviewTime
10000000,5,True,A2744822IV8IQ0,B00FOKZVPU,My wife just loves it and was thrilled.,Five Stars,1447027200
10000001,1,True,A1T74DO8W3GESF,B00FOKZVPU,Not the one I needed so it was returned.,Fire HD 7 Case (old Model),1445817600
10000002,2,True,A31R5W0381P1D3,B00FOKZVPU,The openings are. Of placed to accommodate the...,The openings are. Of placed to accommodate the...,1445558400
10000003,5,True,A2E6PCARIX47TW,B00FOKZVPU,It's fits !,Looks great,1443139200
10000004,5,True,A1YB0WHSNT8BDY,B00FOKZVPU,As described,Five Stars,1442275200
10000005,5,True,A224GZRA2FOIIM,B00FOKZVPU,The item is just as described and works wonder...,Great product,1441584000
10000006,5,True,A3ARDA4ZMK7IR8,B00FOKZVNC,My other Fintie was starting to look shabby af...,... other Fintie was starting to look shabby a...,1498003200
10000007,4,True,A1IJHUGNRGM90H,B00FOKZVNC,Classy looking.,Four Stars,1497484800
10000008,5,True,A394LMFTSQWP63,B00FOKZVNC,Love it.,Five Stars,1496707200
10000009,1,False,A1JP5WW6I4JECX,B00FOKZVNC,This is a really nice looking but its not func...,This is a really nice looking but its not func...,1493424000


### 1.8.4 Home/Kitchen CSV <a name="1.8.4"></a>

In [44]:
hk = pd.read_json("../data/raw/Home_and_Kitchen.json", lines=True, chunksize=1_000_000)

In [46]:
for i, chunk in enumerate(hk):
    df_cleaner(chunk).to_parquet(path=f"../data/edited/kitchen{i}.parquet")

In [48]:
pd.read_parquet("../data/edited/kitchen10.parquet").head(20)

,overall,verified,reviewerID,asin,reviewText,summary,unixReviewTime
11000000,5,True,A2JZREXUGDD9ME,B00K8NKYSC,"Amazing, I always use this for my smoothies in...",A Necessity,1450396800
11000001,5,True,A1ECWTOCBGBC3L,B00K8NKYSC,I was very excited to purchase the certified r...,Great blender!,1450051200
11000002,5,True,A3DRCC09NHCCU1,B00K8NKYSC,Wow! What a great machine. Wish I'd bought one...,Love It!,1449014400
11000003,5,True,A3TI1M63DXABS9,B00K8NKYSC,I'm so glad I got the model with the pre-progr...,Awesome Blender,1448928000
11000004,5,True,A34T5IZTXB956R,B00K8NKYSC,"Smoothies, soups, spreads, raw deserts, ice cr...",we make with this great product. Changed eatin...,1448323200
11000005,5,True,A6FEU436W333L,B00K8NKYSC,Had it over a year and it still work great,Five Stars,1447977600
11000006,5,True,AO7VAMMX2MBEG,B00K8NKYSC,Loving my Vitamix!,Five Stars,1447718400
11000007,5,True,A12TX9Q29R3J26,B00K8NKYSC,i cannot say enough about how much I love my V...,i cannot say enough about how much I love my V...,1447286400
11000008,5,True,A19TEAU5UU0FCE,B00K8NKYSC,Waaayyy so much better than I even thought. Ma...,My Vitamix,1446681600
11000009,5,True,A30JVFW7UK9HWU,B00K8NKYSC,Great machine! We're having fun trying out th...,Start drinking your spinach right now!,1446595200


In [49]:
!ls -ahl ../data/edited

total 7.2G
drwxrwxr-x 3 paul paul 4.0K Apr 14 11:23 .
drwxrwxr-x 5 paul paul 4.0K Apr  5 10:29 ..
-rw-rw-r-- 1 paul paul 245M Apr 14 11:18 electronics0.parquet
-rw-rw-r-- 1 paul paul 178M Apr 14 11:19 electronics10.parquet
-rw-rw-r-- 1 paul paul 163M Apr 14 11:19 electronics11.parquet
-rw-rw-r-- 1 paul paul 166M Apr 14 11:19 electronics12.parquet
-rw-rw-r-- 1 paul paul 177M Apr 14 11:19 electronics13.parquet
-rw-rw-r-- 1 paul paul 171M Apr 14 11:19 electronics14.parquet
-rw-rw-r-- 1 paul paul 172M Apr 14 11:20 electronics15.parquet
-rw-rw-r-- 1 paul paul 173M Apr 14 11:20 electronics16.parquet
-rw-rw-r-- 1 paul paul 214M Apr 14 11:20 electronics17.parquet
-rw-rw-r-- 1 paul paul 204M Apr 14 11:20 electronics18.parquet
-rw-rw-r-- 1 paul paul 158M Apr 14 11:20 electronics19.parquet
-rw-rw-r-- 1 paul paul 260M Apr 14 11:18 electronics1.parquet
-rw-rw-r-- 1 paul paul 159M Apr 14 11:20 electronics20.parquet
-rw-rw-r-- 1 paul paul 256M Apr 14 11:18 electronics2.parquet
-rw-rw-r-- 1 paul paul 

## 1.9 Summary <a name="1.9_summary"></a>

We have cleaned the three datasets. We did not drop any rows from our data yet, as we only had a small number of missing **reviewText** data, which we will explore in the future.

We dropped the **reviewTime** and **reviewerName** columns as they're somewhat duplicated by our **unixReviewTime** and **reviewerID** variables. We dropped the **vote**, **style**, and **image** columns, as they're both largely filled with missing values and unimportant to our analysis generally.

Feature variable: **reviewText** will be our main data source for analysis, though the review summary also contains text, possibly from the user, about their feelings about the item.

Target variable: **overall** stars (1-5) will be converted in the next part of the project into categorical variables, to predict positive, neutral and negative feelings regarding user's opinions of items.